In [ ]:
def rag_model1_():

# Evaluate not contain categori

In [ ]:
#model1
# just only use question data, you don't need to use answer data
#First thing ,put 20 data out from dataset.


#use this 20 data for evaluation
for question in 20_quesiton:
  print(get_similer_question_ranking_5(question))

#model2
# just only use question data, you don't need to use answer data
#First thing ,put 20 data out from dataset.


#use this 20 data for evaluation
for question in 20_quesiton:
  print(get_similer_data_ranking_5(question))

#model3
# just only use question data, you don't need to use answer data
#First thing ,put 20 data out from dataset.


#use this 20 data for evaluation
for question in 20_quesiton:
  print(get_similer_data_ranking_5(question))

#...



query : Is credit card payment available?<br>
<br>
most_similer_data_5 <br>
1 : Do you accept credit card payment?<br>
2 : Do you accept credit card payments?<br>
3 : Do you accept credit card payments?<br>
4 : What payment methods are available?<br>
5 : What are the payment options? <br>

# Evaluate contain categori

In [ ]:
#making rag model wiht categori data.

#model1
# just only use question data, you don't need to use answer data
#First thing ,put 20 data out from dataset.


#use this 20 data for evaluation
for question in 20_quesiton:
  print(get_similer_question_ranking_5(question))

#model2
# just only use question data, you don't need to use answer data
#First thing ,put 20 data out from dataset.


#use this 20 data for evaluation
for question in 20_quesiton:
  print(get_similer_data_ranking_5(question))

#model3
# just only use question data, you don't need to use answer data
#First thing ,put 20 data out from dataset.


#use this 20 data for evaluation
for question in 20_quesiton:
  print(get_similer_data_ranking_5(question))

#...



In [ ]:
!pip install google-cloud-storage
from google.cloud import storage
import pandas as pd

# バケット名とファイル名を指定
bucket_name = 'not_confidencial_data_for_share'
file_name = 'qa_pairs_for_rag_1108.jsonl'
local_file_path = 'qa_pairs_for_rag_1108.jsonl'  # ローカルの保存先

# Storageクライアントを初期化
client = storage.Client()

# バケットとBlobオブジェクトを取得
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(file_name)

# ファイルをローカルにダウンロード
blob.download_to_filename(local_file_path)

In [ ]:
# バケット名とファイル名を指定
bucket_name = 'not_confidencial_data_for_share'
file_name = 'output_with_categories_1107.jsonl'
local_file_path = 'output_with_categories_1107.jsonl'  # ローカルの保存先

# Storageクライアントを初期化
client = storage.Client()

# バケットとBlobオブジェクトを取得
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(file_name)

# ファイルをローカルにダウンロード
blob.download_to_filename(local_file_path)

# **BERT and ScaNN (Model 1)**


In [ ]:
!pip install sentence-transformers scann numpy jsonlines

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 78.7 MB/s eta 0:00:00


With Category

In [ ]:
import jsonlines
import numpy as np
import random
from sentence_transformers import SentenceTransformer
import scann

# Step 1: Load your dataset (output_categories.jsonl)
with jsonlines.open('output_with_categories_1107.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer-category pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']} Category: {pair['category']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]
categories = [pair['category'] for pair in qa_pairs]

# Initialize model for SBERT
model_sbert = SentenceTransformer('all-MiniLM-L6-v2')

# Step 3: Encode the documents using SBERT
question_embeddings = model_sbert.encode(documents, convert_to_numpy=True)

# Step 4: Create ScaNN index for SBERT embeddings
scann_index = scann.scann_ops.builder(question_embeddings, 10, "dot_product").tree(num_leaves=min(1000, len(questions)), num_leaves_to_search=50).score_ah(2, anisotropic_quantization_threshold=0.2).reorder(50).build()

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Step 5: Function to retrieve the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    # Encode the query using SBERT
    query_embedding = model_sbert.encode([f"Question: {query}"], convert_to_numpy=True)

    # Search the ScaNN index
    neighbors, distances = scann_index.search_batched(query_embedding)

    # Return the most similar questions
    closest_questions = [questions[i] for i in neighbors[0][:top_k]]
    return closest_questions

# Evaluate the system with 20 selected questions using SBERT and ScaNN
print("Evaluating using SBERT and ScaNN:")
for question in evaluation_questions:
    similar_questions = find_similar_questions(question)
    print(f"Query: {question}")
    print("Most similar data (top 5):")
    for idx, similar_question in enumerate(similar_questions, start=1):
        print(f"{idx}: {similar_question}")
    print()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Evaluating using SBERT and ScaNN:
Query: 島戸さん宛
Most similar data (top 5):
1: 「島戸さん宛」▼11/3(日)取り組みの報告　・10分読書・3-5課題実施・バナートレース順調に進んでいます👍
2: 「島戸さん宛」▼11/4(月)取り組みの報告　・3-5課題実施・バナートレース・勉強会予約課題でわからないところがあり、勉強会予約しました☺️
3: 島戸さん宛
4: 島戸さんお世話になっております、榎本です。▼今日の取り組み・2-1、2-2　受講・2-2 課題提出
5: 島戸さんこんばんは、榎本です。本日は1つしか行えませんでした。▼本日の報告・3-1 Webページについて

Query: 中川さん宛カリキュラム1-3の再再提出終わりました。
Most similar data (top 5):
1: 中川さん宛カリキュラム1-3の再再提出終わりました。
2: 本日は面談ありがとうございました。カリキュラムを再開し、2-12-2提出しています。ご確認の程、宜しくお願い致します。
3: 藤川様へ【本日取り組んだこと】・3-3 実践課題手直し、再提出・3-4 課題提出
4: 藤川さん3-1.3-2動画視聴と課題提出終わりました！3-3は課題が今途中です！
5: 谷津さんへ4-2 実践課題再提出しました！

Query: 今日は忙しく取り組めません。
Most similar data (top 5):
1: 今日は何も取り組めませんでした。
2: ☁️日次報告☁️今日は取り組めませんでした💦
3: 今日は忙しく取り組めません。
4: ☁️日次報告☁️今日は取り組めませんでした💦
5: 本日も取り組みできませんでした。

Query: こんにちわ！いつもお世話になっております🙇🏼RESUMEの件、確認致しました！代替えサイトにて課題の提出をさせていただきます✨よろしくお願いいたします！
Most similar data (top 5):
1: こんにちわ！いつもお世話になっております🙇🏼RESUMEの件、確認致しました！代替えサイトにて課題の提出をさせていただきます✨よろしくお願いいたします！
2: 5-2.3 提出いたしました
3: お世話になります。week5の③を提出しました。
4: お世話にな

Without Category

In [ ]:
import jsonlines
import numpy as np
import random
from sentence_transformers import SentenceTransformer
import scann

# Step 1: Load your dataset (qa_pairs_for_rag_1108.jsonl)
with jsonlines.open('qa_pairs_for_rag_1108.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]

# Initialize model for SBERT
model_sbert = SentenceTransformer('all-MiniLM-L6-v2')

# Step 3: Encode the documents using SBERT
question_embeddings = model_sbert.encode(documents, convert_to_numpy=True)

# Step 4: Create ScaNN index for SBERT embeddings
scann_index = scann.scann_ops.builder(question_embeddings, 10, "dot_product").tree(num_leaves=min(1000, len(questions)), num_leaves_to_search=50).score_ah(2, anisotropic_quantization_threshold=0.2).reorder(50).build()

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Step 5: Function to retrieve the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    # Encode the query using SBERT
    query_embedding = model_sbert.encode([f"Question: {query}"], convert_to_numpy=True)

    # Search the ScaNN index
    neighbors, distances = scann_index.search_batched(query_embedding)

    # Return the most similar questions
    closest_questions = [questions[i] for i in neighbors[0][:top_k]]
    return closest_questions

# Evaluate the system with 20 selected questions using SBERT and ScaNN
print("Evaluating using SBERT and ScaNN:")
for question in evaluation_questions:
    similar_questions = find_similar_questions(question)
    print(f"Query: {question}")
    print("Most similar data (top 5):")
    for idx, similar_question in enumerate(similar_questions, start=1):
        print(f"{idx}: {similar_question}")
    print()


Evaluating using SBERT and ScaNN:
Query: ありがとうございます。一応、10/17 11:15〜より同質問にて勉強会の予約をさせていただきました。もし、それまでに回答をいただけましたら勉強会は別の質問にさせていただきます。その際の質問(下)Adobe Fireflyについて・立ち上げた時に出て来る画像はファイヤーフライだけ利用して作成しているもの？
Most similar data (top 5):
1: 添削ありがとうございます課題8-3を提出しました8-4もやってみましたが、Fireflyのクレジットが不足していますとなり、生成できませんでした💦
2: 嶋内さんお世話になってます。先日は面談ありがとうございました！次回の面談予定を立てていませんでしたので、ご連絡させて頂きました！返信お待ちしております☺︎
3: こんばんは(月)figmaに取り組んでますが、文字が打てなくなってしまい悩んでます。カーソルは出てるんですが、文字が打ててない状態です。
4: お世話になります。week4-4、5-1、2を提出しました。よろしくお願い致します！5-2、エクセルに関数をコピペしても数字を入力し直さなければ関数が反応しませんでした。互換性の問題でしょうか？
5: ありがとうございます。一応、10/17 11:15〜より同質問にて勉強会の予約をさせていただきました。もし、それまでに回答をいただけましたら勉強会は別の質問にさせていただきます。その際の質問(下)Adobe Fireflyについて・立ち上げた時に出て来る画像はファイヤーフライだけ利用して作成しているもの？

Query: 青木さん宛目標シートを入力しましたので共有させていただきます。ご確認よろしくお願いいたします。
Most similar data (top 5):
1: 青木さんお世話になってます。目標シート記入しました！
2: 青木さん宛本日は２ー２の動画視聴と実践課題を提出しました。
3: 青木さん、こんばんは🌛ご連絡ができておらず申し訳ございません。。。体調を崩し、進捗が遅れてしまいました💦動画視聴は進めており、日中も時間を刻んで進められればと思っています。どうぞよろしくお願いします🙇🏻‍♀️
4: 青木さんお疲れ様です。3-3再提出課題実施バナートレースの勉強会行

# **T5 & FAISS (Model 2)**

In [ ]:
!pip install transformers
!pip install faiss-gpu # For GPU acceleration

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 23.5 MB/s eta 0:00:00


With Category

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import faiss
import jsonlines
import numpy as np
import random

# Step 1: Load your dataset (output_categories.jsonl)
with jsonlines.open('output_with_categories_1107.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer-category pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']} Category: {pair['category']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]
categories = [pair['category'] for pair in qa_pairs]

# Initialize tokenizer and model for T5
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

# Step 3: Function to generate embeddings using T5
def generate_embedding(text):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    with torch.no_grad():
        output = model.encoder(input_ids)
    embeddings = output.last_hidden_state.mean(dim=1).squeeze().numpy()  # Average pooling
    return embeddings

# Step 4: Generate embeddings for each question
question_embeddings = [generate_embedding(doc) for doc in documents]

# Step 5: Initialize Faiss index
d = question_embeddings[0].shape[0]  # Dimensionality of embeddings
faiss_index = faiss.IndexFlatL2(d)
faiss_index.add(np.array(question_embeddings))

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Step 6: Function to retrieve the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    # Generate embedding for the query
    query_embedding = generate_embedding(f"Question: {query}")

    # Search Faiss index for nearest neighbors
    distances, indices = faiss_index.search(query_embedding.reshape(1, -1), top_k)

    # Return the most similar questions
    closest_questions = [questions[i] for i in indices[0]]
    return closest_questions

# Evaluate the system with 20 selected questions using T5 and Faiss
print("Evaluating using T5 and Faiss:")
for question in evaluation_questions:
    similar_questions = find_similar_questions(question)
    print(f"Query: {question}")
    print("Most similar data (top 5):")
    for idx, similar_question in enumerate(similar_questions, start=1):
        print(f"{idx}: {similar_question}")
    print()

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Evaluating using T5 and Faiss:
Query: お世話になります2-3の課題を提出しましたよろしくお願いします
Most similar data (top 5):
1: 課題2-3を提出しました。よろしくお願いします。
2: お世話になります2-3の課題を提出しましたよろしくお願いします
3: すみません。やっぱり提出出来ていませんでした。もう一度提出のためのアドレスを７-3のところに入れていただきたいです。
4: フレームピクセルの指定や画像のピクセルの指定はございますが何でもいいのですか？
5: 度々すみません💦8-2課題提出できました❢まとめての提出で恐縮ですが、よろしくお願いします❢

Query: week7-1,2,3の課題を提出しました。よろしくお願いいたします。
Most similar data (top 5):
1: week3-1.-2,-4実践課題アップいたしました。※添削課題確認お願いいたします。
2: week7-1〜7-3の課題を提出しました
3: Week4-2とweek5-1~4が終了致しました。リンクをコピー致しましたので、ご確認の程宜しくお願い致します。
4: week8-1,2を提出しました。よろしくお願いいたします。
5: week7-3.8-1.8-2の課題を提出しました。

Query: 課題7-3を提出しました
Most similar data (top 5):
1: すみません。やっぱり提出出来ていませんでした。もう一度提出のためのアドレスを７-3のところに入れていただきたいです。
2: こんにちは。7-3の課題を提出しましたのでよろしくお願いします。ただ、プロンプトのリンクの貼り付けをどこにしたらいいのか分からず、欄外に貼り付けました。よろしくお願いします。
3: 課題7-2提出しました
4: 課題7-3を提出しました
5: 度々すみません💦8-2課題提出できました❢まとめての提出で恐縮ですが、よろしくお願いします❢

Query: ご連絡ありがとうございます。本日、入金させていただきます。また、お手数おかけして申し訳ありませんが、8/8あたりに送っていただいた契約書の方をもう一度送っていただくことは可能でしょうか？
Most similar data (top 5):
1: フレー

Without category

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import faiss
import jsonlines
import numpy as np
import random

# Step 1: Load your dataset (qa_pairs_for_rag_1108.jsonl)
with jsonlines.open('qa_pairs_for_rag_1108.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]

# Initialize tokenizer and model for T5
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

# Step 3: Function to generate embeddings using T5
def generate_embedding(text):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    with torch.no_grad():
        output = model.encoder(input_ids)
    embeddings = output.last_hidden_state.mean(dim=1).squeeze().numpy()  # Average pooling
    return embeddings

# Step 4: Generate embeddings for each question
question_embeddings = [generate_embedding(doc) for doc in documents]

# Step 5: Initialize Faiss index
d = question_embeddings[0].shape[0]  # Dimensionality of embeddings
faiss_index = faiss.IndexFlatL2(d)
faiss_index.add(np.array(question_embeddings))

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Step 6: Function to retrieve the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    # Generate embedding for the query
    query_embedding = generate_embedding(f"Question: {query}")

    # Search Faiss index for nearest neighbors
    distances, indices = faiss_index.search(query_embedding.reshape(1, -1), top_k)

    # Return the most similar questions
    closest_questions = [questions[i] for i in indices[0]]
    return closest_questions

# Evaluate the system with 20 selected questions using T5 and Faiss
print("Evaluating using T5 and Faiss:")
for question in evaluation_questions:
    similar_questions = find_similar_questions(question)
    print(f"Query: {question}")
    print("Most similar data (top 5):")
    for idx, similar_question in enumerate(similar_questions, start=1):
        print(f"{idx}: {similar_question}")
    print()


Evaluating using T5 and Faiss:
Query: 遅くなりました、すみません。よろしくお願いします！
Most similar data (top 5):
1: 谷津さま今日の取り組み今日は、病院だった為に制作出来ませんでした。明日からまた取り組みます！
2: 谷津さま今日の取り組み今日は、病院だった為に制作出来ませんでした。明日からまた取り組みます！
3: ありがとうございます！
4: 谷津さま今日の取り組み今日は、病院だった為に制作出来ませんでした。明日からまた取り組みます！
5: 谷津さま今日の取り組み今日は、病院だった為に制作出来ませんでした。明日からまた取り組みます！

Query: 初回ガイダンスについて予約をしようと思います。が、1人の時間がほぼなく子供と一緒でも大丈夫でしょうか？
Most similar data (top 5):
1: 長いのは無理なんですね例えば小説を書いて欲しいとお願いしたい場合一気には無理だけど、1章ずつにしたり、さらに細かく区切って出力してもらい、自分で繋げるんですね
2: 今、追加課題提出されている分が届いていますか？確認お願いします
3: 目標シートの記入をしようとしているのですがこれはパソコンからじゃないと記入できないやつですか？スマホでやろうとしても出来なさそうで、、
4: 課題2できました。
5: 質問があるのですが、各種イベントや自習会は毎回同じ内容でしょうか？自習会は時間がある時は積極的に予約して行った方が良いのでしょうか？

Query: 宜しくお願いします
Most similar data (top 5):
1: 今ワイヤーフレームの修正をおこなっているのですが、「なめらかな〜」「素の唇を〜」「唇に〜」の部分が操作解説動画通りのフォントや文字の大きさにしているのに見本よりも字が濃く･太く見えるのですが、私の勘違いなんでしょうか、、、？
2: ありがとうございます。対応します
3: 課題提出しました。よろしくお願いいたします🙇
4: ☁️日次報告☁️今日は取り組めませんでした💦
5: こんなメッセージが出ました。

Query: 申し訳ございません🙇‍♀️休み間違えてました💦10/9はいかがでしょうか💦
Most similar data (top 5):
1: 申し訳ございませ

# **TF-IDF + Annoy (Model 3)**

In [ ]:
# Install necessary libraries
!pip install scikit-learn annoy jsonlines --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


With category

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from annoy import AnnoyIndex
import jsonlines
import numpy as np
import random

# Step 1: Load your dataset (output_categories.jsonl)
with jsonlines.open('output_with_categories_1107.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer-category pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']} Category: {pair['category']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]
categories = [pair['category'] for pair in qa_pairs]

# Step 3: Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Convert the documents to TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform(documents).toarray()  # Convert to numpy array

# Step 4: Initialize the ANNoy index
dim = tfidf_matrix.shape[1]  # Dimensionality of the vectors (number of features)
annoy_index = AnnoyIndex(dim, 'angular')  # 'angular' is cosine similarity equivalent

# Step 5: Add the TF-IDF vectors to the ANNoy index
for i, vec in enumerate(tfidf_matrix):
    annoy_index.add_item(i, vec)

# Build the index (this step can take a while depending on the number of documents)
annoy_index.build(10)  # 10 trees is a good trade-off for speed and accuracy

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Step 6: Function to retrieve the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    # Convert the query to a TF-IDF vector
    query_tfidf = vectorizer.transform([f"Question: {query}"]).toarray()

    # Get the nearest neighbors from ANNoy
    nearest_neighbors = annoy_index.get_nns_by_vector(query_tfidf[0], top_k, include_distances=False)

    # Return the most similar questions
    closest_questions = [questions[i] for i in nearest_neighbors]
    return closest_questions

# Evaluate the system with 20 selected questions using TF-IDF and ANNoy
print("Evaluating using TF-IDF and ANNoy:")
for question in evaluation_questions:
    similar_questions = find_similar_questions(question)
    print(f"Query: {question}")
    print("Most similar data (top 5):")
    for idx, similar_question in enumerate(similar_questions, start=1):
        print(f"{idx}: {similar_question}")
    print()


Evaluating using TF-IDF and ANNoy:
Query: 川本さん宛てまた次回面談の予約も完了しましたのでご確認お願いいたします。
Most similar data (top 5):
1: 川本さん宛てまた次回面談の予約も完了しましたのでご確認お願いいたします。
2: 【その他】
3: week1の課題を提出しました
4: Week8  8-4の課題を提出しました
5: week6-3, 6-4 課題提出しました。

Query: 準備できておらず申し訳ございませんでしたまたリスケお願いします🙇
Most similar data (top 5):
1: 準備できておらず申し訳ございませんでしたまたリスケお願いします🙇
2: 【その他】
3: week1の課題を提出しました
4: Week8  8-4の課題を提出しました
5: week6-3, 6-4 課題提出しました。

Query: 閲覧権限の変更の仕方がわからず
Most similar data (top 5):
1: 閲覧権限の変更の仕方がわからず
2: 【その他】
3: week1の課題を提出しました
4: Week8  8-4の課題を提出しました
5: week6-3, 6-4 課題提出しました。

Query: 谷津さんへ昨日送り忘れてしまってすみません！昨日3-5の課題を提出し、今日再提出をしました！今日は昼の仕事が忙しかったので、早めに休んで明日また次に進めて行きます！🙇‍♀️
Most similar data (top 5):
1: 谷津さんへ昨日送り忘れてしまってすみません！昨日3-5の課題を提出し、今日再提出をしました！今日は昼の仕事が忙しかったので、早めに休んで明日また次に進めて行きます！🙇‍♀️
2: 昨日3-3動画視聴し、今日は課題に1時間取り組みました！課題は終わらなかったので、また明日頑張ります😃
3: 【その他】
4: week1の課題を提出しました
5: Week8  8-4の課題を提出しました

Query: 【その他】
Most similar data (top 5):
1: 【その他】
2: 【その他】
3: 【その他】
4: 【その他】
5: week1の課題を提出しました

Query: Week2-3③提出しました。
Most si

Without category

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from annoy import AnnoyIndex
import jsonlines
import numpy as np
import random

# Step 1: Load your dataset (qa_pairs_for_rag_1108.jsonl)
with jsonlines.open('qa_pairs_for_rag_1108.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]

# Step 3: Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Convert the documents to TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform(documents).toarray()  # Convert to numpy array

# Step 4: Initialize the ANNoy index
dim = tfidf_matrix.shape[1]  # Dimensionality of the vectors (number of features)
annoy_index = AnnoyIndex(dim, 'angular')  # 'angular' is cosine similarity equivalent

# Step 5: Add the TF-IDF vectors to the ANNoy index
for i, vec in enumerate(tfidf_matrix):
    annoy_index.add_item(i, vec)

# Build the index (this step can take a while depending on the number of documents)
annoy_index.build(10)  # 10 trees is a good trade-off for speed and accuracy

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Step 6: Function to retrieve the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    # Convert the query to a TF-IDF vector
    query_tfidf = vectorizer.transform([f"Question: {query}"]).toarray()

    # Get the nearest neighbors from ANNoy
    nearest_neighbors = annoy_index.get_nns_by_vector(query_tfidf[0], top_k, include_distances=False)

    # Return the most similar questions
    closest_questions = [questions[i] for i in nearest_neighbors]
    return closest_questions

# Evaluate the system with 20 selected questions using TF-IDF and ANNoy
print("Evaluating using TF-IDF and ANNoy:")
for question in evaluation_questions:
    similar_questions = find_similar_questions(question)
    print(f"Query: {question}")
    print("Most similar data (top 5):")
    for idx, similar_question in enumerate(similar_questions, start=1):
        print(f"{idx}: {similar_question}")
    print()


Evaluating using TF-IDF and ANNoy:
Query: 振込完了いたしました。ご確認いただけますようよろしくお願いいたします。
Most similar data (top 5):
1: 振込完了いたしました。ご確認いただけますようよろしくお願いいたします。
2: ありがとうございます！
3: 4-4 実践課題提出しました。
4: 4-2 実践課題提出しました。
5: 5-2 実践課題提出しました

Query: やってみます
Most similar data (top 5):
1: やってみます
2: 川本さん宛て承知いたしました！ありがとうございます✨やってみます！
3: ありがとうございます！
4: 4-4 実践課題提出しました。
5: 4-2 実践課題提出しました。

Query: 島戸さんありがとうございます！まず今回は3-3「ワイヤーフレームの書き方」の課題を行っております。そこで解説動画を参考にさせていただいておりますが、そもそも私の使用しているFigmaのツールバーの表記が異なっています。❶解説動画内で文字幅を自動調整していますが、私のバーにはそれを行える箇所がどこにあるかが不明です。❷またバーの詳細（ex.スタイルを適用）を見ようとすると英字表記になってしまい、日本語に直す事ができません。以上の事で困っています。もし説明の不備等ございましたら申し付けください。
Most similar data (top 5):
1: 島戸さんありがとうございます！まず今回は3-3「ワイヤーフレームの書き方」の課題を行っております。そこで解説動画を参考にさせていただいておりますが、そもそも私の使用しているFigmaのツールバーの表記が異なっています。❶解説動画内で文字幅を自動調整していますが、私のバーにはそれを行える箇所がどこにあるかが不明です。❷またバーの詳細（ex.スタイルを適用）を見ようとすると英字表記になってしまい、日本語に直す事ができません。以上の事で困っています。もし説明の不備等ございましたら申し付けください。
2: ありがとうございます！
3: 4-4 実践課題提出しました。
4: 4-2 実践課題提出しました。
5: 5-2 実践課題提出しました

Query: 2-1から2-3②まで課題完了いたしました。よろしくお願


# Universal Sentence Encoder + HNSW (Model 4)



In [ ]:
!pip install tensorflow tensorflow-hub hnswlib numpy

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=2360795 sha256=d59d46da07dbd2a5f0de8338ab4667a65ce0702eb41c356a9816ce8cdb80d662
  Stored in directory: /root/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib


With category

In [ ]:
import jsonlines
import numpy as np
import random
import tensorflow as tf
import tensorflow_hub as hub
import hnswlib

# Step 1: Load your dataset (output_categories.jsonl)
with jsonlines.open('output_with_categories_1107.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer-category pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']} Category: {pair['category']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]
categories = [pair['category'] for pair in qa_pairs]

# Load Universal Sentence Encoder model from TensorFlow Hub
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Step 3: Encode the documents using USE
question_embeddings = use_model(documents).numpy()

# Step 4: Create HNSW index
dim = question_embeddings.shape[1]
p = hnswlib.Index(space='cosine', dim=dim)
p.init_index(max_elements=len(questions), ef_construction=200, M=16)
p.add_items(question_embeddings)
p.set_ef(50)  # ef should be set to a value between 10 and 100 for best performance

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Step 5: Function to retrieve the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    # Encode the query using USE
    query_embedding = use_model([f"Question: {query}"]).numpy()

    # Search the HNSW index
    labels, distances = p.knn_query(query_embedding, k=top_k)

    # Return the most similar questions
    closest_questions = [questions[i] for i in labels[0]]
    return closest_questions

# Evaluate the system with 20 selected questions using USE and HNSW
print("Evaluating using USE and HNSW:")
for question in evaluation_questions:
    similar_questions = find_similar_questions(question)
    print(f"Query: {question}")
    print("Most similar data (top 5):")
    for idx, similar_question in enumerate(similar_questions, start=1):
        print(f"{idx}: {similar_question}")
    print()


Evaluating using USE and HNSW:
Query: 8-3提出しました
Most similar data (top 5):
1: 6-4.7-3の提出しております。
2: 課題　3-1,3-2を行いました
3: ☁️日次報告☁️今日は取り組めませんでした💦
4: わかりました✨ありがとうございます！🙇‍♀️※返信不要です！
5: お振込でなくカード登録でした。ご確認ください。

Query: 課題3-3.3-4完了しました
Most similar data (top 5):
1: ありがとうございます。
2: 谷津さん、お疲れ様です！当日で大変申し訳無いのですが、急遽仕事が長引いてしまい、面談に間に合いそうにありません(汗)面談をリスケしてもらうことは可能ですか😖？？
3: お世話になります。week3の③,④を提出しました
4: 課題3-3.3-4完了しました
5: 上田さん(emoji)面接の案内ありがとうございます(emoji)明日、よろしくお願いいたします(emoji)

Query: おはようございます実践課題week6-1提出いたしました。よろしくお願いします。
Most similar data (top 5):
1: お疲れ様です！4-4week-実践課題を提出しました。よろしくお願いします(よろしく)
2: 課題　3-1,3-2を行いました
3: 前回4-2.とお伝えしたのは4-3の間違えでした。
4: 入会手続きを完了させていただいていますが、この後のスケジュールはどのようになりますでしょうか。手続き完了からしばらく時間が経過しておりますため再度確認させていただけると幸いです。よろしくお願いします。
5: お世話になっております。土曜日にいただいた受講までに済ませておくことが載っていたメールをたった今作業中に間違えて削除をしてしまいました。大変申し訳ございません。もう一度送信していただくこと可能でしょうか。

Query: 安田さん、本日はありがとうございます。私も趣味でキックボクシング週に3回パーソナルでやってるんで嬉しかったです。また、よろしくお願いします。
Most similar data (top 5):
1: 課題7-3を提出しました
2: すみません、課題の提出を勘違いしていました。先程、課題2-1を提出

Without category

In [ ]:
import jsonlines
import numpy as np
import random
import tensorflow as tf
import tensorflow_hub as hub
import hnswlib

# Step 1: Load your dataset (qa_pairs_for_rag_1108.jsonl)
with jsonlines.open('qa_pairs_for_rag_1108.jsonl') as reader:
    qa_pairs = [obj for obj in reader]

# Step 2: Prepare the question-answer pairs as documents
documents = [f"Question: {pair['question']} Answer: {pair['answer']}" for pair in qa_pairs]
questions = [pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]

# Load Universal Sentence Encoder model from TensorFlow Hub
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Step 3: Encode the documents using USE
question_embeddings = use_model(documents).numpy()

# Step 4: Create HNSW index
dim = question_embeddings.shape[1]
p = hnswlib.Index(space='cosine', dim=dim)
p.init_index(max_elements=len(questions), ef_construction=200, M=16)
p.add_items(question_embeddings)
p.set_ef(50)  # ef should be set to a value between 10 and 100 for best performance

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(questions)), 20)
evaluation_questions = [questions[i] for i in evaluation_indices]

# Step 5: Function to retrieve the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    # Encode the query using USE
    query_embedding = use_model([f"Question: {query}"]).numpy()

    # Search the HNSW index
    labels, distances = p.knn_query(query_embedding, k=top_k)

    # Return the most similar questions
    closest_questions = [questions[i] for i in labels[0]]
    return closest_questions

# Evaluate the system with 20 selected questions using USE and HNSW
print("Evaluating using USE and HNSW:")
for question in evaluation_questions:
    similar_questions = find_similar_questions(question)
    print(f"Query: {question}")
    print("Most similar data (top 5):")
    for idx, similar_question in enumerate(similar_questions, start=1):
        print(f"{idx}: {similar_question}")
    print()


Evaluating using USE and HNSW:
Query: お世話になっております。week2-2とweek2-3を提出しました。よろしくお願いします🙇‍♀️
Most similar data (top 5):
1: ありがとうございます！追加でこのまま質問ですが、こちらのおすすめバナーを練習後、ポートフォリオ作成に向けてオリジナルバナー制作に取り掛かる、で良いでしょうか？
2: ありがとうございます
3: 提出が滞っており申し訳ありません。お疲れ様です！6week-実践課題を提出しました。よろしくお願いします(よろしく)
4: 6-4.7-3の提出しております。
5: お疲れ様です！4-4week-実践課題を提出しました。よろしくお願いします(よろしく)

Query: 樋渡さん遅くなってしまいましたが、2-2の動画視聴をしました。
Most similar data (top 5):
1: 谷津さん11/43-3の提出課題をしました。
2: こんにちは！パソコンを購入したのですが到着が遅れており、まだ手元にきておりません。Slackはパソコンから参加したいので、少しお待ち下さい。よろしくお願いします。
3: 申し訳ありません、まだハローワークの手続きが終わっておらず、途中参加もご迷惑をかけるのでキャンセルさせていただきたいです。
4: Week3-3の課題を提出しました。
5: 失礼しました。15分からでしたので、間に合うかもしれません。

Query: 谷津さんへPC再起動したらできました！🥺✨ありがとうございます！！
Most similar data (top 5):
1: こんなメッセージが出ました。
2: かしこまりました！それまでにパソコンは用意して良いですか？パソコン購入の注意点は入会時に渡された資料を見ればいいですね？
3: 失礼しました。15分からでしたので、間に合うかもしれません。
4: こんにちは。7-3の課題を提出しましたのでよろしくお願いします。ただ、プロンプトのリンクの貼り付けをどこにしたらいいのか分からず、欄外に貼り付けました。よろしくお願いします。
5: こんにちは！パソコンを購入したのですが到着が遅れており、まだ手元にきておりません。Slackはパソコンから参加したいので、少しお待ち下さい。よろしくお願いします


# Word2Vec Implementation (Model 5)



In [ ]:
!pip install nltk
!pip install gensim
!pip install janome  # Install Janome for better tokenization of Japanese text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 77.5 MB/s eta 0:00:00


With category

In [ ]:
import json
import nltk
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from janome.tokenizer import Tokenizer  # For Japanese tokenization
import random

# Initialize Janome tokenizer for Japanese
tokenizer = Tokenizer()

# Function to tokenize Japanese text
def tokenize_japanese(text):
    return [token.surface for token in tokenizer.tokenize(text)]

# Download NLTK data
nltk.download('punkt')

# Load data from JSONL
def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:  # Ensure UTF-8 encoding for Japanese characters
        for line in f:
            data.append(json.loads(line))
    return data

data = load_data('output_with_categories_1107.jsonl')

# Tokenize data using Janome for Japanese text
tokenized_data = []
for item in data:
    question_tokens = tokenize_japanese(item['question'])
    answer_tokens = tokenize_japanese(item['answer'])
    category_tokens = tokenize_japanese(item['category'])
    # Flatten question, answer, and category into the same list for Word2Vec training
    tokenized_data.append(question_tokens + answer_tokens + category_tokens)

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

# Create a vectorized representation of questions, answers, and categories by averaging word vectors
def get_average_vector(tokens):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)  # Average the vectors
    else:
        return None

question_vectors = []
for item in data:
    question_tokens = tokenize_japanese(item['question'])
    question_vec = get_average_vector(question_tokens)
    question_vectors.append(question_vec)

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(data)), 20)
evaluation_questions = [data[i]['question'] for i in evaluation_indices]

# Function to find the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    query_tokens = tokenize_japanese(query)
    query_vec = get_average_vector(query_tokens)

    if query_vec is not None:
        similarities = cosine_similarity([query_vec], [vec for vec in question_vectors if vec is not None])
        most_similar_indices = similarities.argsort()[0][-top_k:][::-1]
        closest_questions = [data[i]['question'] for i in most_similar_indices]
        return closest_questions
    else:
        return []

# Evaluate the system with 20 selected questions
print("Evaluating using Word2Vec:")
for question in evaluation_questions:
    similar_questions = find_similar_questions(question)
    print(f"Query: {question}")
    print("Most similar data (top 5):")
    for idx, similar_question in enumerate(similar_questions, start=1):
        print(f"{idx}: {similar_question}")
    print()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Evaluating using Word2Vec:
Query: ありがとうございます。一応、10/17 11:15〜より同質問にて勉強会の予約をさせていただきました。もし、それまでに回答をいただけましたら勉強会は別の質問にさせていただきます。その際の質問(下)Adobe Fireflyについて・立ち上げた時に出て来る画像はファイヤーフライだけ利用して作成しているもの？
Most similar data (top 5):
1: ありがとうございます。一応、10/17 11:15〜より同質問にて勉強会の予約をさせていただきました。もし、それまでに回答をいただけましたら勉強会は別の質問にさせていただきます。その際の質問(下)Adobe Fireflyについて・立ち上げた時に出て来る画像はファイヤーフライだけ利用して作成しているもの？
2: ＊最終面談について昨夜、メンターの二宮様の予約を本日10/17分でお願いしようとしましたが、時間遅過ぎて、明日10/18の予約しか取れないようでした。明日10/18は終日都合がつきません。その場合、最終面談のご案内に記されていた通り、本日、LINEにて「希望しない」と連絡した方が良いのでしょうか？最後まで、細かい質問となってしまいました。お手数ですが、ご回答よろしくお願いします。
3: 先程イベントのキャリア設計(10月29日)の予約を入れたのですが、補足欄が無かったので担当者に耳が不自由であることをお伝えして頂きたいです😭よろしくお願いいたします。
4: HerTech業務委託
荒井さん、先ほどは勉強会ありがとうございました！最後のご質問「開業届は出さないといけないのか？」について、補足説明させてください。開業届は、個人で事業を開始した場合の義務となっております。（所得税法に定められています。）罰則はありませんが、開業届の提出は任意ではなく、義務です。現実的には、開業届を出さずに事業を行っている者もおりますが、開業届を出して個人の事業をおこなうのが、副業の場合でも正しい姿です。また、開業届を提出すると、所得控除が最大65万円受けられるなどメリットもあります。詳しくは、国税庁のホームページをチェックしてみてください。[https://www.nta.go.jp/publication/pamph/koho/kurashi/html

In [ ]:
import json
import nltk
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from janome.tokenizer import Tokenizer  # For Japanese tokenization
import random

# Initialize Janome tokenizer for Japanese
tokenizer = Tokenizer()

# Function to tokenize Japanese text
def tokenize_japanese(text):
    return [token.surface for token in tokenizer.tokenize(text)]

# Download NLTK data
nltk.download('punkt')

# Load data from JSONL
def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:  # Ensure UTF-8 encoding for Japanese characters
        for line in f:
            data.append(json.loads(line))
    return data

data = load_data('qa_pairs_for_rag_1108.jsonl')

# Tokenize data using Janome for Japanese text
tokenized_data = []
for item in data:
    question_tokens = tokenize_japanese(item['question'])
    answer_tokens = tokenize_japanese(item['answer'])
    # Flatten question and answer into the same list for Word2Vec training
    tokenized_data.append(question_tokens + answer_tokens)

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

# Create a vectorized representation of questions by averaging word vectors
def get_average_vector(tokens):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)  # Average the vectors
    else:
        return None

question_vectors = []
for item in data:
    question_tokens = tokenize_japanese(item['question'])
    question_vec = get_average_vector(question_tokens)
    question_vectors.append(question_vec)

# Select 20 random questions for evaluation
evaluation_indices = random.sample(range(len(data)), 20)
evaluation_questions = [data[i]['question'] for i in evaluation_indices]

# Function to find the 5 most similar questions based on query
def find_similar_questions(query, top_k=5):
    query_tokens = tokenize_japanese(query)
    query_vec = get_average_vector(query_tokens)

    if query_vec is not None:
        similarities = cosine_similarity([query_vec], [vec for vec in question_vectors if vec is not None])
        most_similar_indices = similarities.argsort()[0][-top_k:][::-1]
        closest_questions = [data[i]['question'] for i in most_similar_indices]
        return closest_questions
    else:
        return []

# Evaluate the system with 20 selected questions
print("Evaluating using Word2Vec:")
for question in evaluation_questions:
    similar_questions = find_similar_questions(question)
    print(f"Query: {question}")
    print("Most similar data (top 5):")
    for idx, similar_question in enumerate(similar_questions, start=1):
        print(f"{idx}: {similar_question}")
    print()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Evaluating using Word2Vec:
Query: 登録完了しました！よろしくお願いいたします☺️
Most similar data (top 5):
1: 登録完了しました！よろしくお願いいたします☺️
2: 出来ました！よろしくお願いします！
3: 確認致しました‼️手続きいたします。
4: 嶋内さん目標シートできました！よろしくお願いします🙇‍♀️
5: 課題提出しました。よろしくお願いいたします🙇

Query: Week4-4の課題を提出しました。
Most similar data (top 5):
1: Week4-4の課題を提出しました。
2: Week3-1の課題を提出しました。
3: week6-4の課題を提出しました。
4: Week5-1の課題を提出しました。
5: Week5-3の課題を提出しました。

Query: 【受講料と入会金】
Most similar data (top 5):
1: 【受講料と入会金】
2: 【受講開始後の解約】
3: 【受講キャンセル・解約】
4: 【返金制度】
5: 【お支払いについて】

Query: 先程4-1の課題を提出したのですがまた誤って4-2の方に提出してしまいました……後ほど提出するので、削除していただけると幸いです……申し訳ありません。
Most similar data (top 5):
1: 先程4-1の課題を提出したのですがまた誤って4-2の方に提出してしまいました……後ほど提出するので、削除していただけると幸いです……申し訳ありません。
2: 先程4-1の課題を提出したのですがまた誤って4-2の方に提出してしまいました……後ほど提出するので、削除していただけると幸いです……申し訳ありません。
3: 先程4-1の課題を提出したのですがまた誤って4-2の方に提出してしまいました……後ほど提出するので、削除していただけると幸いです……申し訳ありません。
4: 添削ありがとうございます課題8-3を提出しました8-4もやってみましたが、Fireflyのクレジットが不足していますとなり、生成できませんでした💦
5: すみません。やっぱり提出出来ていませんでした。もう一度提出のためのアドレスを７-3のところに入れていただきたいです。

Query: 課題2できました。
Most 

# agent builder
https://console.cloud.google.com/gen-app-builder/engines?hl=ja&project=monobrain-development

In [ ]:
#same process as other 5 model's

# Evaluate the system with 20 selected questions
print("Evaluating using Word2Vec:")
for question in evaluation_questions:
    similar_questions = find_similar_questions(question)
    print(f"Query: {question}")
    print("Most similar data (top 5):")
    for idx, similar_question in enumerate(similar_questions, start=1):
        print(f"{idx}: {similar_question}")
    print()